In [1]:
import numpy as np
import pandas as pd
d=pd.read_csv('data_files/Had_role.csv')

In [2]:
d = d.fillna("NULL")

In [3]:
import pymysql
mydb = pymysql.connect(
    host='localhost',
    user='root',
    passwd='221928122',
    database='imdb')

In [8]:
cursor = mydb.cursor()
sql = "select database();"
cursor.execute(sql)
record = cursor.fetchone()
for df in batches(d,10000):
    my_data = []
    for i,row in df.iterrows():
        my_data.append(tuple(row))
    sql = "INSERT INTO imdb.had_role VALUES (%s,%s,%s)"
    cursor.executemany(sql,my_data)
    print("Records inserted")
    mydb.commit()

In [5]:
def batches(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [28]:
mydb.close()

In [5]:
d

,title_id,name_id,role_
12299,tt0045907,nm0695033,Lupita
12300,tt0045907,nm0622843,Juan Godínez 'Caireles'
12301,tt0045907,nm0815524,Tobías Hernández 'Tarrajas'
12302,tt0045907,nm0411600,Papá Pinillos
12303,tt0045908,nm0209956,Arturo Giorgi
...,...,...,...
81825,tt0061998,nm0001419,Jerrie
81826,tt0062000,nm0010721,Inanimate
81827,tt0062000,nm0038775,House Maid
81828,tt0062000,nm0067635,Watchman #3


In [2]:
d = d.dropna()

In [9]:
import numpy as np
import pandas as pd
import os
import gzip
import shutil


def unzip_files(folder):


    out_files = []

    # For each file in folder
    for file in os.listdir(folder):

        # If it is gzipped unzip it
        if file.endswith(".gz"):

            in_file_path = os.path.join(folder, file)

            out_file_path = os.path.join(folder,file.replace('.gz',''))

            out_files.append(out_file_path)

            print('\tUnzipping ',in_file_path,' to ',out_file_path)

            # Unzip current file
            with gzip.open(in_file_path, 'rb') as f_in:
                with open(out_file_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

    return out_files


def make_Aliases(title_akas):

    Aliases = pd.DataFrame()
    Aliases[['title_id','ordering','title','region','language',
    'is_original_title']] = title_akas[['titleId','ordering','title','region','language',
    'isOriginalTitle']]
    Aliases.to_csv('Aliases.csv',index=False)

def make_Alias_types(title_akas):

    Alias_types = pd.DataFrame()
    Alias_types[['title_id','ordering','type']] = title_akas[['titleId','ordering','types']]
    Alias_types = Alias_types.dropna()
    Alias_types.to_csv('Alias_types.csv',index=False)


def make_Alias_attributes(title_akas):

    Alias_attributes = pd.DataFrame() 
    Alias_attributes[['title_id','ordering','attribute']] =  title_akas[['titleId','ordering','attributes']]
    Alias_attributes = Alias_attributes.dropna()
    Alias_attributes.to_csv('Alias_attributes.csv',index=False)


def make_Directors_and_Writers(title_crew):

    Directors = pd.DataFrame()
    Directors[['title_id','name_id']] = title_crew[['tconst','directors']]
    Writers[['title_id','name_id']] = title_crew[['tconst','writers']]

    Directors = Directors.dropna()
    Writers = Writers.dropna()

    Directors = Directors.assign(name_id=Directors.name_id.str.split(',')).explode('name_id').reset_index(drop=True)
    Writers = Writers.assign(name_id=Writers.name_id.str.split(',')).explode('name_id').reset_index(drop=True)

    Directors.to_csv('Directors.csv',index=False)
    Writers.to_csv('Writers.csv',index=False)


def make_Episode_belongs_to(title_episode):
    Episode_belongs_to = title_episode.rename(columns={
    'tconst':'title_id',
    'parentTconst':'parent_tv_show_title_id',
    'seasonNumber':'season_number',
    'episodeNumber':'episode_number'
    })

    Episode_belongs_to.to_csv('Episode_belongs_to.csv',index=False)


def make_Names_(name_basics):
    Names_[['name_id','name_','birth_year','death_year']] = name_basics[['nconst','primaryName','birthYear','deathYear']]
    Names_.to_csv('Names_.csv',index=False)

def make_Name_worked_as(name_basics):
    Name_worked_as[['name_id','profession']] = name_basics[['nconst','primaryProfession']]
    Name_worked_as = Name_worked_as.dropna()
    Name_worked_as = Name_worked_as.assign(profession=Name_worked_as.profession.str.split(',')).explode('profession').reset_index(drop=True)
    Name_worked_as.to_csv('Name_worked_as.csv',index=False)


def make_Known_for(name_basics):


    Known_for[['name_id','title_id']] = name_basics[['nconst','knownForTitles']]
    Known_for = Known_for.dropna()
    Known_for = Known_for.assign(title_id=Known_for.title_id.str.split(',')).explode('title_id').reset_index(drop=True)
    Known_for.to_csv('Known_for.csv',index=False)


def make_Principals(title_principals):
    Principals[['title_id','ordering','name_id','job_category','job']] = title_principals[['tconst','ordering','nconst','category','job']]
    Principals.to_csv('Principals.csv',index=False)

def make_Had_role(title_principals):
    Had_role[['title_id','name_id','role_']] = title_principals[['tconst','nconst','characters']]
    Had_role = Had_role.dropna()

    Had_role['role_'] = Had_role['role_'].str.replace('[\"\[\]]','',regex=True)
    Had_role['role_'] = Had_role['role_'].str.replace('\\','|')
    Had_role = Had_role.assign(role_=Had_role.role_.str.split(',')).explode('role_').reset_index(drop=True)
    Had_role['role_'] = Had_role['role_'].str.title()
    Had_role['role_'] = Had_role['role_'].str.replace('^ | $','',regex=True)
    Had_role.drop_duplicates(keep=False,inplace=True)
    Had_role.to_csv('Had_role.csv',index=False)


def create_movies_series_titles(title_basics):
    Titles[['title_id','title_type','primary_title',
    'original_title', 'is_adult', 'start_year', 'end_year', 'runtime_minutes']] = title_basics[['tconst','titleType',
    'primaryTitle','originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes']]
    Titles.to_csv('Titles.csv',index=False)


def create_title_genres(title_basics):

    Title_genres[['title_id','genre']] = title_basics[['tconst','genres']]

    Title_genres = Title_genres.dropna()

    Title_genres = Title_genres.assign(genre=Title_genres.genre.str.split(',')).explode('genre').reset_index(drop=True)

    Title_genres.to_csv('Title_genres.csv',index=False)


def make_Title_ratings(title_ratings):

    Title_ratings = title_ratings.rename(columns={
    'tconst':'title_id',
    'averageRating':'average_rating',
    'numVotes':'num_votes'
    })
    Title_ratings.to_csv('Title_ratings.csv',index=False)

    

data_path = './imdb_data'
print('Looking for IMDb data in: ',data_path,'\n')



data_files = unzip_files(data_path)

print('\n','Reading title.akas.tsv ...','\n')
title_akas = pd.read_csv(os.path.join(data_path,'title_akas_2.csv'))

make_Aliases(title_akas)
make_Alias_types(title_akas)
make_Alias_attributes(title_akas)

del title_akas

print('\n','Reading title.crew.tsv','\n')

title_crew = pd.read_csv(os.path.join(data_path,'title_crew_2.csv'))

make_Directors_and_Writers(title_crew)

del title_crew


print('\n','Reading title.episode.tsv ...','\n')

title_episode = pd.read_csv(os.path.join(data_path,'title_episode_2.csv'))

make_Episode_belongs_to(title_episode)
# Delete title_episode
del title_episode


print('\n','Reading name.basics.tsv ...','\n')

name_basics  = pd.read_csv(os.path.join(data_path,'name_basics_2.csv'))

make_Names_(name_basics)
make_Name_worked_as(name_basics)
make_Known_for(name_basics)

del name_basics

print('\n','Reading title.principals.tsv ...','\n')

title_principals = pd.read_csv(os.path.join(data_path,'title_principals_2.csv'))

make_Principals(title_principals)
make_Had_role(title_principals)

del title_principals

title_basics = pd.read_csv(os.path.join(data_path,'title_basics_2.csv'))

make_Titles(title_basics)
make_Title_genres(title_basics)

del title_basics

print('\n','Reading title.ratings.tsv ...','\n')

title_ratings = pd.read_csv(os.path.join(data_path,'title_ratings_2.csv'))

make_Title_ratings(title_ratings)

del title_ratings

Looking for IMDb data in:  ./imdb_data 

	Unzipping  ./imdb_data\name.basics.tsv.gz  to  ./imdb_data\name.basics.tsv


KeyboardInterrupt: 